In [1]:
from __future__ import print_function, division
import os,sys
import numpy as np
import torch # pytorch package, allows using GPUs
# fix seed
seed=17
np.random.seed(seed)
torch.manual_seed(seed)

In [15]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import datasets # load data
from torch.nn.utils.rnn import pad_sequence

all_data = 'ExpL project/MPSEQCN1.csv'
#all_data = 'ExpL project/MP_W_Seq.csv'
# Mapping: assign unique numbers (1-20) based on polarity order (least polar → most polar)
aa_to_polarity = {
    'L': 1,  # Leucine
    'F': 2,  # Phenylalanine
    'I': 3,  # Isoleucine
    'C': 4,  # Cysteine
    'M': 5,  # Methionine
    'V': 6,  # Valine
    'W': 7,  # Tryptophan
    'Y': 8,  # Tyrosine
    'P': 9,  # Proline
    'A': 10, # Alanine
    'T': 11, # Threonine
    'G': 12, # Glycine
    'S': 13, # Serine
    'H': 14, # Histidine
    'Q': 15, # Glutamine
    'R': 16, # Arginine
    'K': 17, # Lysine
    'N': 18, # Asparagine
    'E': 19, # Glutamic acid
    'D': 20  # Aspartic acid
}

def sequence_to_polarity_numbers(seq, mapping=aa_to_polarity):
    """
    Convert an amino acid sequence to a list of unique integer values based on polarity.
    Amino acids not in the mapping will be assigned 0.
    """
    return [mapping.get(aa, 0) for aa in seq]

Code notes:

Defining the dataset (Silkome_Dataset)
    
    - the class declaration will load the data from the all_data csv file and process it into a Pytorch format

__init__:
    
    - this initializes the dataset by taking the file path, data_type (ie train or test data), transform, test_size, and random_state (for reproducibility)
   
    - it reads the CSV file into a pd dataframe and then extracts the individual IDs from the first column (0)
    
    - it then extracts the mechanical properties from indexes 11, 13, 15, and 17, and converts it into a NumPy array, they are in 32-bit float format for easier numerical operations
   
    - the next part extracts and cleans up the sequence data, it extracts column 21 and replaces missing values with empty strings and converts it into a NumPy array
   
    - it loops through each row of sequence data and splits the string using ',' as the delimiter, it gets rid of empty strings
   
   
    - it also filters by length, just in case anything is longer or shorter than what the predetermined min and max lengths are, and it only allows a max of 2 sequences per ID, this can be changed as some have up to 8 sequences

Train/Test split:
   
    - data is split into training and testing data sets

__len__:
   
    - returns the number of samples in the dataset, pytorch uses this to know how many samples are available

__getitem__:
   
    - this method retrieves a sample at a certain index and retrieves the ID, the mechanical properties, and the amino acid sequence(s)
    it returns a dictionary with the keys: 'id', 'mechanical_props', and 'sequences'

custom_collate:
   
    - this handles the variable length sequences by converting the mechanical props into a tensor while keeping the IDs and sequences as lists
    
    - this was done due to the error I recieved from pytorch expecting every item in a batch to have the same shape, due to the variable lengths of the sequences, a collate function was used

load_data:
    
    - creates two data sets for training and testing, it wraps each dataset into a dataloader that will batch the data, shuffle the training data (not the test), use the collate function to handle batch size issues and returns the training and testing dataloaders

__name__:
   
    - example usage
    
    - prints the IDs, mechanical properties as a tensor, and the sequences
    

In [16]:
class Silkome_Dataset(Dataset):
    """Silkome pytorch dataset for the mechanical properties and amino acid sequences.
     Assumes:
      - uses a CSV file where:
          - index 0: Individual IDs.
          - indexes 11, 13, 15, 17: Mechanical properties 
            (in order: "toughness", "youngs_modulus", "tensile_strength", "strain_at_break").
          - index 21: Amino acid sequences concatenated from two columns.
            (Sequences are assumed to be separated by a comma ',')
      - Each cell in the sequence column must contain at least one sequence.
      - Maximum number of sequences per cell is 2.
      - Each sequence must have a length between 151 and 1515.
    """

    def __init__(self, all_data, data_type='train', transform=None, test_size=0.2, random_state=42):
        """
       Args:
            all_data (str): Path to the CSV file. (ExpL project/MP_W_Seq.csv)
            data_type (str): 'train' or 'test'. Use 'train' for the training set and 'test' for the testing set.
            transform (callable, optional): Optional transform to be applied on a sample.
            test_size (float): Fraction of data to use as test set.
            random_state (int): Seed for splitting.
        """
        self.df = pd.read_csv(all_data)
        
        # Extract individual IDs from column 0
        self.ids = self.df.iloc[:, 0].values

        # Extract mechanical properties from columns 11, 13, 15, 17
        self.mechanical_props = self.df.iloc[:, [11, 13, 15, 17]].values.astype(np.float32)

        # Process the amino acid sequences from column 21.
        # First, fill missing values (if any) with an empty string.
        raw_sequences = self.df["MaSp1"].fillna('').values

        processed_sequences = []
        polarity_numeric_sequences = []  # Store numeric representations based on polarity.
        for i, s in enumerate(raw_sequences):
            # Split on comma; adjust the delimiter if needed.
            seq_list = [seq.strip() for seq in s.split(',') if seq.strip() != '']
            
            # Filter each sequence by length (151 to 1515 AAs)
            valid_seqs = [seq for seq in seq_list if 151 <= len(seq) <= 1515]
            #print (valid_seqs)
            
            processed_sequences.append(valid_seqs)
            
            # Convert each valid sequence to its numeric representation based on polarity.
            numeric_seq_list = [sequence_to_polarity_numbers(seq) for seq in valid_seqs]
            polarity_numeric_sequences.append(numeric_seq_list)
        #print("Length of IDs:", len(self.ids))
        #print("Length of processed sequences:", len(processed_sequences))
        #print("DataFrame shape:", self.df.shape)
        #print("Columns:", self.df.columns)
        #print("Raw sequences array length:", len(raw_sequences))
        #print("First few raw sequences:", raw_sequences[:5])

        
        self.sequences = processed_sequences
        self.polarity_numeric_sequences = polarity_numeric_sequences
        
        #train/test split
        if data_type in ['train', 'test']:
            indices = np.arange(len(self.ids))
            train_idx, test_idx = train_test_split(indices, test_size=test_size, random_state=random_state)
            if data_type == 'train':
                sel = train_idx
                print("Using training set (80% of data)")
            else:
                sel = test_idx
                print("Using testing set (20% of data)")
            
            self.ids = self.ids[sel]
            self.mechanical_props = self.mechanical_props[sel]
            self.sequences = [self.sequences[i] for i in sel]
            self.polarity_numeric_sequences = [self.polarity_numeric_sequences[i] for i in sel]
        
        self.transform = transform

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        sample = {
            'id': self.ids[idx],
            'mechanical_props': self.mechanical_props[idx],  # ex, a numpy array of shape (4,)
            'sequences': self.sequences[idx],  # list of one to eight sequences (strings)
            'polarity_numeric_sequences': self.polarity_numeric_sequences[idx]
        }
        if self.transform:
            sample = self.transform(sample)
        return sample

def custom_collate(batch):
    """
    Custom collate function to handle variable-length sequences.
    It converts mechanical properties to a tensor while keeping the IDs as a list and sequences as a list of lists.
    """
    batch_ids = [item['id'] for item in batch]
    # Convert mechanical_props into a tensor
    props_list = [torch.tensor(item['mechanical_props'], dtype=torch.float) for item in batch]
    batch_props = pad_sequence(props_list, batch_first=True, padding_value=0)
    # Keep sequences as list-of-lists (each inner list can be of different length)
    batch_sequences = [item['sequences'] for item in batch]
    batch_polarity_numeric_seqs = [item['polarity_numeric_sequences'] for item in batch]
    return {
        'id': batch_ids, 
        'mechanical_props': batch_props, 
        'sequences': batch_sequences,
        'polarity_numeric_sequences': batch_polarity_numeric_seqs
    }

def load_data(all_data, batch_size, test_batch_size, **kwargs):
    """
    Create PyTorch DataLoaders for training and testing from the CSV file.
    
    Args:
        all_data (str): Path to the CSV file.
        batch_size (int): Batch size for the training loader.
        test_batch_size (int): Batch size for the test loader.
        **kwargs: Additional arguments for DataLoader (e.g., num_workers, pin_memory).
    
    Returns:
        train_loader, test_loader: DataLoader objects.
    """
    train_dataset = Silkome_Dataset(all_data=all_data, data_type='train')
    test_dataset = Silkome_Dataset(all_data=all_data, data_type='test')
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                              collate_fn=custom_collate, **kwargs)
    test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, 
                             collate_fn=custom_collate, **kwargs)
    
    return train_loader, test_loader

# Example usage:
if __name__ == "__main__":
    csv_path = 'ExpL project/MPSEQCN1.csv'
    train_loader, test_loader = load_data(csv_path, batch_size=32, test_batch_size=32)
    
    # Iterate through one batch from the training loader and print a sample
    for batch in train_loader:
        print("IDs:", batch['id'])
        print("Mechanical properties:", batch['mechanical_props'])
        print("Sequences:", batch['sequences'])
        print("Polarity numeric sequences:", batch['polarity_numeric_sequences'])
        break

[['...']]
Using training set (80% of data)
[['...']]
Using testing set (20% of data)
IDs: [3534, 3860, 3656, 1894, 5086, 5058, 1974, 5268, 5286, 7303, 5118, 4039, 3868, 7392, 5290, 3819, 7324, 5775, 3900, 4912, 5364, 4959, 5803, 3590, 3875, 6796, 4910, 5352, 5199, 5146, 7447, 4275]
Mechanical properties: tensor([[1.0600e-01, 1.0100e+01, 9.7000e-01, 1.6700e+01],
        [8.7000e-02, 2.4100e+01, 1.6800e+00, 8.7000e+00],
        [5.0000e-02, 6.3300e+00, 7.0000e-01, 1.3100e+01],
        [5.0000e-02, 9.6000e+00, 6.3000e-01, 1.2100e+01],
        [8.4000e-02, 2.1600e+00, 1.2100e+00, 2.3300e+01],
        [1.1100e-01, 2.3400e+00, 9.2000e-01, 2.9800e+01],
        [2.3000e-02, 6.9900e+00, 5.0000e-01, 8.1000e+00],
        [9.1000e-02, 3.1400e+00, 1.1800e+00, 1.9100e+01],
        [2.4100e-01, 1.2800e+01, 1.5600e+00, 2.7600e+01],
        [1.8100e-01, 8.7800e+00, 1.6300e+00, 2.2500e+01],
        [3.4000e-02, 2.5700e+00, 5.4000e-01, 1.4400e+01],
        [1.2600e-01, 3.2100e+00, 7.7000e-01, 3.5400e+01]

Defining the Neural Network and its Architecture

In [61]:
import torch.nn as nn # construct NN
import torch.nn.functional as F

class Model(nn.Module):
    # create convolutional net
    def __init__(self, input_dim=4, hidden_dim=64, output_dim=4):
        """
        Args:
            input_dim (int): Number of input features (mechanical properties), default is 4.
            hidden_dim (int): Number of neurons in the hidden layer.
            output_dim (int): Number of output classes (e.g., for classification).
        """
        # inherit attributes and methods of nn.Module
        super(Model, self).__init__()	
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        # create convolutional layer
        self.conv1 = nn.Conv2d(in_channels=num_features, out_channels=64, kernel_size=3, padding=1)
        # batch norm layer takes Depth
        self.bn1=nn.BatchNorm2d(64) 
        pooled_len = max_len // 4
        # create fully connected layer after maxpool operation reduced 40->18
        self.fc1 = nn.Linear(128 * pooled_len, 256) 	
        self.N=max_len
        self.L=num_classes
        print("The number of neurons in CNN layer is %i"%(max_len))

    def forward(self, x):
        """
        Forward pass for regression.
        
        Args:
            sample (dict): A dictionary with at least the key 'mechanical_props' that contains 
                           a tensor of shape [batch_size, input_dim].
        
        Returns:
            A tensor of shape [batch_size, output_dim] containing the predicted continuous values.
        """
        
        x= x['mechanical_props']
        #Unsqueeze command indicates one channel and turns x.shape from (:,40,40) to (:,1, 40,40)
        x=F.relu(self.fc1(x))
        #print(x.shape)  often useful to look at shapes for debugging
        x = self.fc2(x)
        return x

# Example usage:
if __name__ == "__main__":
    # Example dummy input (batch of 32 samples, each with 4 features)
    model = Model()
    dummy_input = {'mechanical_props': torch.randn(32, 4)}
    predictions = model(dummy_input)
    print("Predictions shape:", predictions.shape)  # Expected shape: [32, 1]

The number of neurons in CNN layer is 1515


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x4 and 48384x256)

In [16]:
def train(epoch):
    # these are very standard functions for going over data to train

    CNN.train() # effects Dropout and BatchNorm layers
    for batch_idx, (data, target) in enumerate(train_loader):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = CNN(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(data_loader,verbose='Test'):
    # these are very standard functions for evaluating data

    CNN.eval() # effects Dropout and BatchNorm layers
    test_loss = 0
    correct = 0
    for data, target in data_loader:
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        output = CNN(data)
        test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()

    test_loss /= len(data_loader.dataset)
    print('\n'+verbose+' set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))
    accuracy=100. * correct / len(data_loader.dataset)
    return(accuracy)


In [17]:
import argparse # handles arguments
import sys; sys.argv=['']; del sys # required to use parser in jupyter notebooks

# training settings
parser = argparse.ArgumentParser(description='PyTorch Convmodel Ising Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
args = parser.parse_args()
args.epochs=5
args.cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

cuda_kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}

In [32]:
import torch.nn.functional as F # implements forward and backward definitions of an autograd operation
import torch.optim as optim # different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc

# load data
train_loader, test_loader = load_data(all_data, batch_size=32, test_batch_size=32, collate_fn=custom_collate)

test_array=[]
critical_array=[]

# create array of depth of convolutional layer
N_array=[1,5,10,20,50]

# loop over depths
for N in N_array:
    CNN_model = CNN()
    if args.cuda:
        CNN_model = CNN_model.cuda()

    # negative log-likelihood (nll) loss for training: takes class labels NOT one-hot vectors!
    criterion = F.nll_loss
    # define SGD optimizer
    optimizer = optim.SGD(CNN_model.parameters(), lr=args.lr, momentum=args.momentum)
    #optimizer = optim.Adam(DNN.parameters(), lr=0.001, betas=(0.9, 0.999))

    # train the CNN and test its performance at each epoch
    for epoch in range(1, args.epochs + 1):
        train(epoch)
        if epoch==args.epochs:
            test_array.append(test(test_loader,verbose='Test'))
            critical_array.append(test(critical_loader,verbose='Critical'))
        else:
            test(test_loader,verbose='Test')
            test(critical_loader,verbose='Critical')
    print(test_array)
    print(critical_array)

Using training set (80% of data)
Using testing set (20% of data)


NameError: name 'sequences' is not defined

In [19]:
from matplotlib import pyplot as plt

## Print the result for different N
%matplotlib inline

plt.plot(N_array, test_array, 'r-*', label="test")
plt.plot(N_array, critical_array, 'b-s', label="critical")
plt.ylim(60,110)
plt.xlabel('Depth of hidden layer', fontsize=24)
plt.xticks(N_array)
plt.ylabel('Accuracy', fontsize=24)
plt.legend(loc='best', fontsize=24)
plt.tick_params(axis='both', which='major', labelsize=24)
plt.tight_layout()
plt.show()

ValueError: x and y must have same first dimension, but have shapes (5,) and (0,)